# **Deep learning for dynamic network analysis (DLDNA) - Final project**
**Dolphins:** R. ARNAUD M. DELPLANQUE A. KARILA-COHEN A. RAMPOLDI <br> **Dataset:** Comprehensive soil classification dataset: https://www.kaggle.com/datasets/ai4a-lab/comprehensive-soil-classification-datasets/code

_______________

# Notebook 4 — Comparative Analysis of Deep Learning Architectures

This notebook consolidates and compares the results obtained from all previous experiments, providing a global assessment of the proposed deep learning approaches for soil classification.

> ### Compared Approaches
> The following models and training strategies are evaluated:
> - **CNN (baseline)**: Convolutional neural network trained on the original dataset.
> - **CNN + cGAN**: CNN trained with data augmented using a Conditional GAN.
> - **CNN + StarGAN**: CNN trained with multi-domain augmented data generated by StarGAN.
> - **Transformer (baseline)**: Swin Transformer leveraging self-attention mechanisms for global feature extraction.
> - **Transformer + GAN augmentation**: Transformer trained using GAN- or StarGAN-augmented datasets.
>
> ### Evaluation Criteria
> Models are compared using the following metrics and analyses:
> - Overall classification accuracy,
> - Confusion matrices for class-wise performance,
> - Generalization behavior on the test set,
> - Sensitivity to class imbalance and minority classes.

### Objective
The objectives of this comparative study are to:
- Identify the most effective architecture for soil image classification,
- Assess the impact of generative data augmentation strategies,
- Analyze trade-offs between model complexity, performance, and robustness.

### Outcome
The results presented in this notebook form the basis for the final discussion and conclusions of the report, highlighting the strengths and limitations of each approach in a geotechni


______

# Model Comparison Results

This notebook compares the performance of different models:
- **CNN**: Simple Convolutional Neural Network
- **Transformer**: Swin Transformer (Attention Mechanism)
- **GAN**: CNN with Conditional GAN Data Augmentation
- **StarGAN**: Results from StarGAN experiments

_____

## 1. Setup and Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import re
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

## 2. Load Results from All Models

In [ ]:
# Define paths
project_root = Path('..')
results_root = project_root / 'results'
comparison_root = results_root / 'comparison'
comparison_root.mkdir(parents=True, exist_ok=True)

# Define model folders
model_folders = {
    'CNN': results_root / 'CNN',
    'Transformer': results_root / 'Attention',
    'GAN': results_root / 'GAN',
    'StarGAN': results_root / 'StarGAN'
}

# Check which models have results
available_models = {}
for model_name, folder_path in model_folders.items():
    if folder_path.exists() and any(folder_path.iterdir()):
        available_models[model_name] = folder_path
        print(f"✓ {model_name}: Results found at {folder_path}")
    else:
        print(f"✗ {model_name}: No results found at {folder_path}")

print(f"\nTotal models with results: {len(available_models)}")

✗ CNN: No results found at ..\models\CNN
✗ Transformer: No results found at ..\models\Attention
✗ GAN: No results found at ..\models\GAN
✗ StarGAN: No results found at ..\models\StarGAN

Total models with results: 0


## 3. Parse Performance Metrics from Text Files

In [3]:
def parse_evaluation_report(file_path):
    """Parse evaluation report to extract key metrics."""
    metrics = {}
    
    if not file_path.exists():
        return None
    
    with open(file_path, 'r') as f:
        content = f.read()
    
    # Try to extract accuracy
    accuracy_match = re.search(r'[Tt]otal [Aa]ccuracy[:\s]+(\d+\.?\d*)%?', content)
    if accuracy_match:
        metrics['accuracy'] = float(accuracy_match.group(1)) / 100 if float(accuracy_match.group(1)) > 1 else float(accuracy_match.group(1))
    
    # Try to extract precision
    precision_match = re.search(r'[Mm]acro [Pp]recision[:\s]+(\d+\.?\d*)', content)
    if precision_match:
        metrics['precision'] = float(precision_match.group(1))
    
    # Try to extract recall
    recall_match = re.search(r'[Mm]acro [Rr]ecall[:\s]+(\d+\.?\d*)', content)
    if recall_match:
        metrics['recall'] = float(recall_match.group(1))
    
    # Try to extract F1
    f1_match = re.search(r'[Mm]acro [Ff]1[- ][Ss]core[:\s]+(\d+\.?\d*)', content)
    if f1_match:
        metrics['f1_score'] = float(f1_match.group(1))
    
    # Also try weighted average from classification report
    weighted_match = re.search(r'weighted avg\s+(\d+\.\d+)\s+(\d+\.\d+)\s+(\d+\.\d+)', content)
    if weighted_match and not metrics:
        metrics['precision'] = float(weighted_match.group(1))
        metrics['recall'] = float(weighted_match.group(2))
        metrics['f1_score'] = float(weighted_match.group(3))
    
    return metrics if metrics else None

# Collect metrics from all models
all_metrics = {}

for model_name, folder_path in available_models.items():
    # Try different possible filenames
    possible_files = [
        folder_path / 'evaluation_report.txt',
        folder_path / 'overall_metrics.txt',
        folder_path / 'metrics.txt',
        folder_path / 'results.txt'
    ]
    
    for file_path in possible_files:
        metrics = parse_evaluation_report(file_path)
        if metrics:
            all_metrics[model_name] = metrics
            print(f"✓ {model_name}: Metrics loaded from {file_path.name}")
            break
    
    if model_name not in all_metrics:
        print(f"✗ {model_name}: Could not find or parse metrics file")

print(f"\nSuccessfully loaded metrics for {len(all_metrics)} models")


Successfully loaded metrics for 0 models


In [4]:
# Display collected metrics
if all_metrics:
    metrics_df = pd.DataFrame(all_metrics).T
    metrics_df = metrics_df.sort_values('accuracy', ascending=False) if 'accuracy' in metrics_df.columns else metrics_df
    print("\n" + "="*70)
    print("MODEL PERFORMANCE COMPARISON")
    print("="*70)
    print(metrics_df.to_string())
    print("="*70)
else:
    print("No metrics could be loaded. Please check if evaluation reports exist.")

No metrics could be loaded. Please check if evaluation reports exist.


## 4. Visualize Performance Comparison

In [5]:
if all_metrics and len(all_metrics) > 0:
    # Create comparison bar plots
    metrics_to_plot = ['accuracy', 'precision', 'recall', 'f1_score']
    available_metrics = [m for m in metrics_to_plot if m in metrics_df.columns]
    
    if available_metrics:
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        axes = axes.flatten()
        
        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
        
        for idx, metric in enumerate(available_metrics):
            if idx < len(axes):
                ax = axes[idx]
                data = metrics_df[metric].sort_values(ascending=False)
                bars = ax.bar(range(len(data)), data.values, color=colors[:len(data)])
                ax.set_xticks(range(len(data)))
                ax.set_xticklabels(data.index, rotation=45, ha='right')
                ax.set_ylabel('Score')
                ax.set_title(f'{metric.replace("_", " ").title()}', fontsize=14, fontweight='bold')
                ax.set_ylim([0, 1.0])
                ax.grid(axis='y', alpha=0.3)
                
                # Add value labels on bars
                for bar in bars:
                    height = bar.get_height()
                    ax.text(bar.get_x() + bar.get_width()/2., height,
                           f'{height:.3f}',
                           ha='center', va='bottom', fontsize=10, fontweight='bold')
        
        # Remove extra subplots if any
        for idx in range(len(available_metrics), len(axes)):
            fig.delaxes(axes[idx])
        
        plt.suptitle('Model Performance Comparison', fontsize=16, fontweight='bold', y=1.00)
        plt.tight_layout()
        
        # Save figure
        output_path = comparison_root / 'model_comparison_metrics.png'
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        print(f"\nComparison plot saved to {output_path}")
        plt.show()
else:
    print("Not enough data to create comparison plots")

Not enough data to create comparison plots


## 5. Radar Chart Comparison

In [6]:
if all_metrics and len(all_metrics) > 1:
    # Create radar chart
    from math import pi
    
    available_metrics = [m for m in ['accuracy', 'precision', 'recall', 'f1_score'] if m in metrics_df.columns]
    
    if len(available_metrics) >= 3:
        # Number of variables
        categories = [m.replace('_', ' ').title() for m in available_metrics]
        N = len(categories)
        
        # Create angles for each metric
        angles = [n / float(N) * 2 * pi for n in range(N)]
        angles += angles[:1]
        
        # Initialize plot
        fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))
        
        # Plot each model
        for idx, (model_name, row) in enumerate(metrics_df.iterrows()):
            values = [row[m] for m in available_metrics]
            values += values[:1]
            
            ax.plot(angles, values, 'o-', linewidth=2, label=model_name, color=colors[idx % len(colors)])
            ax.fill(angles, values, alpha=0.15, color=colors[idx % len(colors)])
        
        # Fix axis to go from 0 to 1
        ax.set_ylim(0, 1)
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(categories, size=12)
        ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
        ax.set_yticklabels(['0.2', '0.4', '0.6', '0.8', '1.0'], size=10)
        ax.grid(True)
        
        plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=12)
        plt.title('Model Performance Radar Chart', size=16, fontweight='bold', pad=20)
        
        # Save figure
        output_path = comparison_root / 'model_comparison_radar.png'
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        print(f"Radar chart saved to {output_path}")
        plt.show()

## 6. Load and Compare Confusion Matrices

In [7]:
# Display confusion matrices side by side
confusion_matrices = {}

for model_name, folder_path in available_models.items():
    cm_path = folder_path / 'confusion_matrix.png'
    if cm_path.exists():
        confusion_matrices[model_name] = cm_path

if confusion_matrices:
    from PIL import Image
    
    n_models = len(confusion_matrices)
    cols = min(2, n_models)
    rows = (n_models + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(12*cols, 10*rows))
    if n_models == 1:
        axes = [axes]
    else:
        axes = axes.flatten() if n_models > 1 else [axes]
    
    for idx, (model_name, cm_path) in enumerate(confusion_matrices.items()):
        img = Image.open(cm_path)
        axes[idx].imshow(img)
        axes[idx].set_title(f'{model_name} Confusion Matrix', fontsize=14, fontweight='bold')
        axes[idx].axis('off')
    
    # Remove extra subplots
    for idx in range(len(confusion_matrices), len(axes)):
        fig.delaxes(axes[idx])
    
    plt.tight_layout()
    
    # Save combined figure
    output_path = comparison_root / 'all_confusion_matrices.png'
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    print(f"\nCombined confusion matrices saved to {output_path}")
    plt.show()
else:
    print("No confusion matrices found")

No confusion matrices found


## 7. Training Curves Comparison (if available)

In [8]:
# Display training curves side by side
training_curves = {}

for model_name, folder_path in available_models.items():
    curve_path = folder_path / 'training_curves.png'
    if curve_path.exists():
        training_curves[model_name] = curve_path

if training_curves:
    from PIL import Image
    
    n_models = len(training_curves)
    cols = min(2, n_models)
    rows = (n_models + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(14*cols, 6*rows))
    if n_models == 1:
        axes = [axes]
    else:
        axes = axes.flatten() if n_models > 1 else [axes]
    
    for idx, (model_name, curve_path) in enumerate(training_curves.items()):
        img = Image.open(curve_path)
        axes[idx].imshow(img)
        axes[idx].set_title(f'{model_name} Training Curves', fontsize=14, fontweight='bold')
        axes[idx].axis('off')
    
    # Remove extra subplots
    for idx in range(len(training_curves), len(axes)):
        fig.delaxes(axes[idx])
    
    plt.tight_layout()
    
    # Save combined figure
    output_path = comparison_root / 'all_training_curves.png'
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    print(f"Combined training curves saved to {output_path}")
    plt.show()
else:
    print("No training curves found")

No training curves found


## 8. Summary Report

In [9]:
# Generate comprehensive summary report
if all_metrics:
    report_path = comparison_root / 'comparison_summary.txt'
    
    with open(report_path, 'w') as f:
        f.write("="*70 + "\n")
        f.write("COMPREHENSIVE MODEL COMPARISON REPORT\n")
        f.write("="*70 + "\n\n")
        
        f.write("Models Evaluated:\n")
        f.write("-" * 70 + "\n")
        for model_name in all_metrics.keys():
            f.write(f"  - {model_name}\n")
        f.write("\n")
        
        f.write("Performance Metrics:\n")
        f.write("-" * 70 + "\n")
        f.write(metrics_df.to_string())
        f.write("\n\n")
        
        # Best performing model per metric
        f.write("Best Performing Models:\n")
        f.write("-" * 70 + "\n")
        for metric in metrics_df.columns:
            best_model = metrics_df[metric].idxmax()
            best_score = metrics_df[metric].max()
            f.write(f"  {metric.replace('_', ' ').title():20s}: {best_model:15s} ({best_score:.4f})\n")
        
        f.write("\n")
        f.write("="*70 + "\n")
        
        # Overall recommendation
        if 'accuracy' in metrics_df.columns:
            best_overall = metrics_df['accuracy'].idxmax()
            f.write(f"\nRECOMMENDATION: {best_overall} shows the best overall accuracy.\n")
        
        f.write("="*70 + "\n")
    
    print(f"\nComprehensive summary report saved to {report_path}")
    
    # Display the report
    with open(report_path, 'r') as f:
        print("\n" + f.read())
else:
    print("No metrics available to generate summary report")

No metrics available to generate summary report


## 9. Export Results to CSV

In [10]:
# Save metrics to CSV for further analysis
if all_metrics:
    csv_path = comparison_root / 'model_comparison.csv'
    metrics_df.to_csv(csv_path)
    print(f"\nMetrics exported to {csv_path}")
    print("\nYou can now use this CSV file for further analysis or reporting.")
else:
    print("No metrics to export")

No metrics to export
